# Cuaderno 27: Formulación de planos cortantes para el ATSP
## (con separación de soluciones fraccionarias)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo dirigido **completo** $D=(V,A)$; y,
* un vector $c \in \ZZ^{A}$ de costos asociados a los arcos de $D$.

El *problema del agente viajero asimétrico (Asymmetric Traveling Salesman Problem, ATSP)* consiste en encontrar un circuito dirigido que visite **todos** los nodos de $G$ y que tenga el menor costo posible.

Vamos a considerar una formulación como programa lineal entero para el ATSP que emplea desigualdades de cortes para eliminar subciclos (ver Cuaderno 25 para el caso simétrico):

\begin{align*}
\min &\sum_{(i,j) \in A} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{(i,j) \in \delta^{+}(i) } x_{ij} = 1, \quad \forall i \in V,\\
&\sum_{(j,i) \in \delta^{-}(i) } x_{ji} = 1, \quad \forall i \in V,\\
&\sum_{ij \in \delta^{+}(W)} x_{ij} \geq 1, \quad \forall W \subset V, \emptyset \neq W \neq V,\\
& x_{ij} \in \{0, 1\}, \quad \forall (i,j) \in A,
\end{align*}
donde $\delta^{+}(W) := \{ (i,j) \in A \, : \, i \in W, j \not\in W\}$.

En este cuaderno vamos a implementar un algoritmo de planos cortantes para separar las restricciones de corte, basado en el algoritmo de corte de capacidad mínima presentado en el Cuaderno 28.


Definimos primero los datos. Usaremos la función `randint` del módulo random para generar valores aleatorios en el rango {0,..,100} para las coordenadas de cada nodo. Los costos de los arcos serán iguales a las distancias euclideanas entre sus nodos extremos. 

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import random as rm
import math 

# iniciar generador de numeros aleatorios
rm.seed(0)

# Numero de nodos del grafo
n = 50

# Nodos del grafo
V = gp.tuplelist(range(1,n+1))

# Posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = gp.tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
A = c.keys()


Emplearemos el módulo `matplotlib` para graficar el tour de la solución. Definiremos para ello la función `dibujarTour` que recibe tres argumentos: una lista `coordx` con las coordenadas horizontales de los nodos, una lista `coordy` con las coordenadas verticales y un vector `tour` con una permutación de los nodos indicando el orden de visita en la solución.

In [ ]:
import matplotlib.pyplot as plt
import random

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour]
    Ty = [coordy[i] for i in tour]
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
        plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=2)
    plt.show()


Para la generación dinámica de las desigualdades de corte, necesitaremos calcular el corte saliente de capacidad mínima en un grafo dirigido. Emplearemos para ello la siguiente función descrita en el Cuaderno 27b:

In [ ]:
import networkx as nx

# corte saliente de capacidad minima
def corte_sal_minimo(V, u):
    # Parametros:
    # V: lista con los nodos del grafo
    # u: diccionario indexado por los arcos del grafo, con sus capacidades
    # se fija un nodo s arbitrario
    s = V[0]
    V2 = [i for i in V if i!=s]
    G = nx.DiGraph()
    G.add_weighted_edges_from([(i, j, u[i,j]) for (i,j) in u.keys()], weight='u')
    # inicializar dmin con la suma de capacidades de todos los arcos
    dmin = sum([u[i,j] for (i, j) in u.keys()])
    hay_solucion = False
    # se calculan los cortes minimos (s, t) y (t, s) para todo t != s
    for t in V2: 
        d1, (W1, W1c) = nx.minimum_cut(G, s, t, capacity='u')
        d2, (W2, W2c) = nx.minimum_cut(G, t, s, capacity='u')
        if d1 < d2 and d1 < dmin:
            dmin, Wmin, Wminc = d1, W1, W1c
            hay_solucion = True
        elif d2 <= d1 and d2 < dmin:
            dmin, Wmin, Wminc = d2, W2, W2c
            hay_solucion = True
    if  not hay_solucion:
        print('*** Error:')
        print(V)
        for (i,j) in [(i, j) for (i, j) in u.keys() if i==1]:
            print([(1,j), u[1,j]])
    return dmin, Wmin, Wminc


Definimos ahora el objeto modelo, las variables y la función objetivo. 

Definimos las variables miembro `_x`, `_V` y `_A` dentro del objeto modelo para tener acceso a las variables de selección de arcos, la lista de nodos y la lista de arcos, respectivamente. 

In [ ]:
# Crear el objeto modelo
m = gp.Model('atsp-corte-lazy')

# Crear las variables
x = m.addVars(A, name="x", vtype=GRB.BINARY)

# Crear la funcion objetivo
m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

# Crear variables en el objeto modelo para tener acceso a x, V y A
m._x = x
m._V = V
m._A = A

Añadimos las restricciones de grado. 

In [ ]:
# Restricciones de grado saliente
m.addConstrs((x.sum(i,'*') == 1 for i in V), "g_saliente");

# Restricciones de grado entrante
m.addConstrs((x.sum('*', i) == 1 for i in V), "g_entrante");



Definimos una función callback para separar dinámicamente las restricciones de corte. Esta función actúa cada vez que el solver encuentra una nueva solución entera que satisface las restricciones de grado (`where == GRB.Callback.MIPSOL`), o cuando termina el procesamiento de un nodo del árbol de branch-and-bound y se dispone de una solución fraccionaria a la relajación lineal problema (`where == GRB.Callback.MIPNODE`). 

Cuando se dispone de una solución entera, la función accede a los valores de las variables a través del llamado a `cbGetSolution`. A partir de estos valores, la función implementa un algoritmo de marcaje para determinar el conjunto W de todos los nodos que pueden ser alcanzados desde el nodo 1. Si este conjunto es distinto de V, el mismo está asociado a una desigualdad de corte violada por `x`.

Cuando la función callback es llamada luego del procesamiento de un nodo del árbol de branch-and-bound (`where == GRB.Callback.MIPNODE`), es necesario en primer lugar verificar que se dispone de una solución fraccionaria, llamando a la función `cbGet(GRB.Callback.MIPNODE_STATUS)` y comparando su valor de retorno con `GRB.OPTIMAL`. En este caso, los valores de la solución fraccionaria son recuperados llamando a la función `cbGetNodeRel`. Estos valores se usan como capacidades en los arcos del grafo del problema. Se emplean las funciones descritas en el Cuaderno 28 para calcular un corte de capacidad mínimima en el grafo. Si la capacidad de este corte es inferior a 1, se separa la desigualdad correspondiente.

In [ ]:
# Funcion callback para separar desigualdades de corte
def mycallback(model, where):
    # Esta funcion se activara cuando se encuentre una nueva solucion entera
    if where == GRB.Callback.MIPSOL:
        # Recuperar los valores de la solucion actual
        vx = model.cbGetSolution(model._x)
        # Determinar los arcos seleccionados en la solucion
        L = gp.tuplelist([(i,j) for (i,j) in model._A if vx[i,j]>0.1])
        # Construir la lista W de nodos que pueden ser alcanzados desde 1
        W = [1]
        i = 1
        while True :
            # seleccionar el único arco saliente de i en L
            a = L.select(i,'*')[0]
            L.remove(a)
            if a[1]==1: 
                break
            W.append(a[1])
            i = a[1]
        # Si W!=V, agregar la desigualdad de corte asociada a W
        if len(W)!=len(model._V):
            Wc = [i for i in model._V if i not in W]
            model.cbLazy(model._x.sum(W, Wc) >= 1)
    # Esta funcion se activara cuando se encuentre la solucion optima en un nodo
    elif where == GRB.Callback.MIPNODE:
        if model.cbGet(GRB.Callback.MIPNODE_STATUS) == GRB.OPTIMAL:
            # Recuperar los valores de la solucion (fraccionaria) actual
            vx = model.cbGetNodeRel(model._x)
            # Crear diccionario de capacidades (=valores de x) indexado por los arcos
            D={}
            for (i,j) in model._A:
                D[i,j] = vx[i,j]
            # encontrar el corte saliente de capacidad minima
            u, W, Wc = corte_sal_minimo(model._V, D)
            # Si la capacidad de este corte es inferior a 1, agregar nueva desigualdad lazy
            if u <= 0.99:
                model.cbLazy(model._x.sum(W, Wc) >= 1)


Escribimos el modelo a un archivo de texto:

In [ ]:
# Escribir el modelo a un archivo
m.write('atsp-corte-lazy.lp')


Para poder utilizar restricciones tipo *lazy* en un modelo, debe fijarse el parámetro `LazyConstraints` al valor de 1.

In [ ]:
# Configurar Gurobi para usar restricciones lazy
m.Params.LazyConstraints = 1


Fijamos un límite para el tiempo de cálculo y una tolerancia para la brecha de optimalidad:

In [ ]:
# Terminar al alcanzar un Gap del 10%
m.Params.MIPGap = 0.1

# Terminar luego de 180 segundos
m.Params.TimeLimit = 180

Finalmente, resolvemos el modelo y mostramos la solución, en caso de que se haya encontrado una solución factible (si `m.SolCount` es mayor a cero).

In [ ]:
# Se recomienda llamar a update() antes de resolver un modelo que usa restricciones lazy
m.update()

# Calcular la solución óptima
m.optimize(mycallback)

# Escribir la solución
if m.SolCount > 0:
    # Recuperar los valores de las variables
    vx = m.getAttr('x', x)
    print('\nTour optimo:')
    for i,j in A:
        if vx[i,j] > 0:
            print('{} --> {}'.format(i, j))


Si encontramos alguna solución factible, el siguiente fragmento de código nos permite graficarla.

In [ ]:
# Crear lista con arcos seleccionados en la solucion
vx = m.getAttr('x', x)
L = gp.tuplelist([(i,j) for i,j in A if vx[i,j]>0.1])
# print(L)

# Recuperar el tour como un ordenamiento de los nodos
T = [1]
# nodo actual:
i = 1
while True:
    # Determinar sucesor de i
    a = L.select(i,'*')[0]
    L.remove(a)
    # Colocar sucesor en la lista del tour y actualizar i
    T.append(a[1])
    i = a[1]
    # Terminar cuando el nodo colocado sea 1
    if i==1: 
        break;
        
# Graficar el tour
dibujarTour(coordx, coordy, T)    

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementación de modelos lineales enteros
# Problema del agente viajero asimétrico (ATSP)
# Implementación con desigualdades lazy

# Luis M. Torres (EPN 2022)

import gurobipy as gp
from gurobipy import GRB
import random as rm
import math 
import networkx as nx

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour]
    Ty = [coordy[i] for i in tour]
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
        plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=2)
    plt.show()

# Funciona para calcular un corte saliente de capacidad minima
def corte_sal_minimo(V, u):
    # Parametros:
    # V: lista con los nodos del grafo
    # u: diccionario indexado por los arcos del grafo, con sus capacidades
    # se fija un nodo s arbitrario
    s = V[0]
    V2 = [i for i in V if i!=s]
    G = nx.DiGraph()
    G.add_weighted_edges_from([(i, j, u[i,j]) for (i,j) in u.keys()], weight='u')
    # inicializar dmin con la suma de capacidades de todos los arcos
    dmin = sum([u[i,j] for (i, j) in u.keys()])
    hay_solucion = False
    # se calculan los cortes minimos (s, t) y (t, s) para todo t != s
    for t in V2: 
        d1, (W1, W1c) = nx.minimum_cut(G, s, t, capacity='u')
        d2, (W2, W2c) = nx.minimum_cut(G, t, s, capacity='u')
        if d1 < d2 and d1 < dmin:
            dmin, Wmin, Wminc = d1, W1, W1c
            hay_solucion = True
        elif d2 <= d1 and d2 < dmin:
            dmin, Wmin, Wminc = d2, W2, W2c
            hay_solucion = True
    if  not hay_solucion:
        print('*** Error:')
        print(V)
        for (i,j) in [(i, j) for (i, j) in u.keys() if i==1]:
            print([(1,j), u[1,j]])
    return dmin, Wmin, Wminc

# Funcion callback para separar desigualdades de corte
def mycallback(model, where):
    # Esta funcion se activara cuando se encuentre una nueva solucion entera
    if where == GRB.Callback.MIPSOL:
        # Recuperar los valores de la solucion actual
        vx = model.cbGetSolution(model._x)
        # Determinar los arcos seleccionados en la solucion
        L = gp.tuplelist([(i,j) for (i,j) in model._A if vx[i,j]>0.1])
        # Construir la lista W de nodos que pueden ser alcanzados desde 1
        W = [1]
        i = 1
        while True :
            # seleccionar el único arco saliente de i en L
            a = L.select(i,'*')[0]
            L.remove(a)
            if a[1]==1: 
                break
            W.append(a[1])
            i = a[1]
        # Si W!=V, agregar la desigualdad de corte asociada a W
        if len(W)!=len(model._V):
            Wc = [i for i in model._V if i not in W]
            model.cbLazy(model._x.sum(W, Wc) >= 1)
    # Esta funcion se activara cuando se encuentre la solucion optima en un nodo
    elif where == GRB.Callback.MIPNODE:
        if model.cbGet(GRB.Callback.MIPNODE_STATUS) == GRB.OPTIMAL:
            # Recuperar los valores de la solucion (fraccionaria) actual
            vx = model.cbGetNodeRel(model._x)
            # Crear diccionario de capacidades (=valores de x) indexado por los arcos
            D={}
            for (i,j) in model._A:
                D[i,j] = vx[i,j]
            # encontrar el corte saliente de capacidad minima
            u, W, Wc = corte_sal_minimo(model._V, D)
            # Si la capacidad de este corte es inferior a 1, agregar nueva desigualdad lazy
            if u <= 0.99:
                model.cbLazy(model._x.sum(W, Wc) >= 1)
    
# iniciar generador de numeros aleatorios
rm.seed(0)

# Numero de nodos del grafo
n = 50

# Nodos del grafo
V = gp.tuplelist(range(1,n+1))

# Posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = gp.tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
A = c.keys()

try:
    # Crear el objeto modelo
    m = gp.Model('atsp-corte-lazy')

    # Crear las variables
    x = m.addVars(A, name="x", vtype=GRB.BINARY)

    # Crear la funcion objetivo
    m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

    # Crear variables en el objeto modelo para tener acceso a x, V y A
    m._x = x
    m._V = V
    m._A = A

    # Restricciones de grado saliente
    m.addConstrs((x.sum(i,'*') == 1 for i in V), "g_saliente");

    # Restricciones de grado entrante
    m.addConstrs((x.sum('*', i) == 1 for i in V), "g_entrante");

    # Escribir el modelo a un archivo
    # m.write('atsp-corte-lazy.lp')

    # Configurar Gurobi para usar restricciones lazy
    m.Params.LazyConstraints = 1

    # Terminar al alcanzar un Gap del 10%
    m.Params.MIPGap = 0.1

    # Terminar luego de 180 segundos
    m.Params.TimeLimit = 180

    # Se recomienda llamar a update() antes de resolver un modelo que usa restricciones lazy
    m.update()

    # Calcular la solucion optima
    m.optimize(mycallback)

    # Escribir la solucion
    if m.SolCount > 0:
        # Recuperar los valores de las variables
        vx = m.getAttr('x', x)
        print('\nTour optimo:')
        for i,j in A:
            if vx[i,j] > 0:
                print('{} --> {}'.format(i, j))

        # Recuperar el tour como un ordenamiento de los nodos
        L = gp.tuplelist([(i,j) for i,j in A if vx[i,j]>0.1])
        T = [1]
        i = 1
        while True:
            # Determinar sucesor de i
            a = L.select(i,'*')[0]
            L.remove(a)
            # Colocar sucesor en la lista del tour y actualizar i
            T.append(a[1])
            i = a[1]
            # Terminar cuando el nodo colocado sea 1
            if i==1: 
                break
        # Graficar el tour
        dibujarTour(coordx, coordy, T)        
        
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')